<a href="https://colab.research.google.com/github/giacomogreggio/HSL-citybikes-predictor/blob/master/HSL_citybikes_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Citybike predictor

### Elevator pitch
Scheduling your day is important for everyone, but every day we have to face problems related to planning your itinerary. When you want to use a citybike to move from a place to another you may find yourself at an empty bike station. Could there be a way to predict the availability? A solution: an application that predicts exactly that based on the time and the weather.


### Data: sources, wrangling, management		
- The original purpose of the data is not compatible with our needs: the data is meant to describe bike trips/routing, not the bike availability
- 
            
### Data analysis: statistics, machine learning	
- We need a predicting model
- Predictions for time series: a lot of different variables
- Combining different data sources to base the prediction to current situation: weather, time of the day, current bike availability    


### Communication of results: summarization & visualization
- Finding clear and intuitive way to summarize and visualize data such that it is accessible to the user
- 
            
### Operationalization: creating added value, end-user point of view
- Mobile optimated web application

## Preprocessing the HSL-data

### Initializing everything

In [2]:
!pip install mpld3

     |████████████████████████████████| 1.0MB 2.5MB/s 
  Created wheel for mpld3: filename=mpld3-0.5.1-cp36-none-any.whl size=364064 sha256=0185ac3bd483c6b5facaec1ea8ad36494eb7017053645de0f6bcae430b59e6dd
  Stored in directory: /root/.cache/pip/wheels/38/68/06/d119af6c3f9a2d1e123c1f72d276576b457131b3a7bf94e402
Successfully built mpld3


In [1]:
# All imports

import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from datetime import datetime
import mpld3
from mpld3 import plugins
mpld3.enable_notebook()

In [4]:
drive.mount('/content/drive')


Mounted at /content/drive


### The function that processes the data for a month

In [2]:
def get_station_data():
  stations = pd.read_csv("/content/drive/My Drive/HSLDataset/Helsingin_ja_Espoon_kaupunkipyöräasemat.csv")
  stations = stations.drop(["FID", "Nimi", "Namn", "Adress", "Kaupunki", "Stad", "Operaattor"], axis = 1)
  return stations

In [12]:
def preprocess_month(filepath):
  data = pd.read_csv(filepath, sep = ",")

  # Make time a datetime object to ease handling. Also floor to starting hour
  data["Dep date"] = pd.to_datetime(data["Departure"], errors = "ignore").dt.floor(freq = "H")
  data["Return date"] = pd.to_datetime(data["Return"], errors = "ignore").dt.floor(freq = "H")

  # For our analysis we shouldn't need this information
  data = data.drop(columns=["Covered distance (m)", "Duration (sec.)", "Departure", "Return"])

  # Get the outgoing bikes per station at timeframe
  outgoing = data.groupby("Departure station id")["Dep date"].value_counts()
  outgoing = outgoing.sort_index()
  outgoing = outgoing.rename_axis(index = {"Dep date" : "Date", "Departure station id" : "ID"})
  outgoing = outgoing.rename("Outgoing")

  # Get the arriving bikes per station at timeframe
  arriving = data.groupby("Return station id")["Return date"].value_counts()
  arriving = arriving.sort_index()
  arriving = arriving.rename_axis(index = {"Return date" : "Date", "Return station id" : "ID"})
  arriving = arriving.rename("Arriving")

  outgoing_arriving_merge = pd.merge(outgoing, arriving, on = ["ID", "Date"], how = "outer")
  outgoing_arriving_merge = outgoing_arriving_merge.fillna(0)

  # We need data for ALL timeframes
  stations = set(outgoing_arriving_merge.index.get_level_values(0))
  all_dates = pd.date_range("2019-04-01 00:00", "2019-04-30 23:00", freq = "H")
  idx = pd.MultiIndex.from_product([stations, all_dates], names = ["ID", "Date"])
  mega_frame_with_station_date_cartesian_product = pd.DataFrame(index = idx)
  processed = pd.merge(mega_frame_with_station_date_cartesian_product, outgoing_arriving_merge, on = ["ID", "Date"], how = "left")
  processed = processed.fillna(0)
  processed = processed.reset_index()

  # Merge with the station data from HSL
  station_data = get_station_data()
  processed_with_station_data = pd.merge(processed, station_data, on = "ID", how = "inner")

  return processed_with_station_data

In [20]:
def get_processed_data_for_month(month):
  month = '{:02.0f}'.format(month)
  data = pd.read_csv("/content/drive/My Drive/HSLDataset/processed/2019-" + month + "-processed.csv")
  data["Date"] = pd.to_datetime(data["Date"])
  
  # Don't know what this is all about but I guess everything is fine-ish :DDD
  data = data.drop("Unnamed: 0", axis = 1)
  return data

In [13]:
# Process all the data and save them as csv-files
path = "/content/drive/My Drive/HSLDataset/od-trips-2019/"
extension = ".csv"

for month in range(4,11):
  filename = "2019-" + '{:02.0f}'.format(month)
  full_path = path + filename + extension
  data = preprocess_month(full_path)
  data.to_csv("./drive/My Drive/HSLDataset/processed/" + filename + "-processed.csv")

## Looking at the data

In [10]:
def data_of_station_for_weekdays_in_month(dataframe, station, month, weekday):
  station_data = dataframe[dataframe["ID"] == station]
  station_data_for_month = station_data[(station_data["Date"].dt.month == month) & (station_data["Date"].dt.weekday == weekday)]
  return station_data_for_month

In [18]:
data = get_processed_data_for_month(4)

## Different data visualizations

In [13]:
wanted_data = data_of_station_for_weekdays_in_month(data, 20, 4, 5)
fig, axs = plt.subplots(1, 4, figsize = (30,6))
saturdays = set(wanted_data["Date"].dt.date)

for idx,saturday in enumerate(saturdays):
  saturday_data = wanted_data[wanted_data["Date"].dt.date == saturday]
  axs[idx].grid(True, alpha = 0.3)
  axs[idx].plot(saturday_data["Date"].dt.hour, saturday_data["Outgoing"], 'r', saturday_data["Date"].dt.hour, saturday_data["Arriving"], 'b', alpha = 0.3)
  axs[idx].title.set_text(saturday)
mpld3.display()